In [ ]:
%pip install fastdwt

In [ ]:
# Define Trajectory 1
trajectory1 = [
    {"lat": 40.747776, "lng": -73.987974, "t": "2024-04-09T11:00:00Z"},
    {"lat": 40.748776, "lng": -73.986974, "t": "2024-04-09T11:05:00Z"},
    {"lat": 40.749776, "lng": -73.985974, "t": "2024-04-09T11:10:00Z"},
    {"lat": 40.750776, "lng": -73.984974, "t": "2024-04-09T11:15:00Z"},
]

trajectory2 = [
    {"lat": 40.747876, "lng": -73.988074, "t": "2024-04-09T11:00:00Z"},
    {"lat": 40.748876, "lng": -73.987074, "t": "2024-04-09T11:05:00Z"},
    {"lat": 40.749876, "lng": -73.986074, "t": "2024-04-09T11:10:00Z"},
    {"lat": 40.750876, "lng": -73.985074, "t": "2024-04-09T11:15:00Z"},
]

trajectory3 = [
    {"lat": 40.747976, "lng": -73.988174, "t": "2024-04-09T11:00:00Z"},
    {"lat": 40.748976, "lng": -73.987174, "t": "2024-04-09T11:05:00Z"},
    {"lat": 40.749976, "lng": -73.986174, "t": "2024-04-09T11:10:00Z"},
    {"lat": 40.750976, "lng": -73.985174, "t": "2024-04-09T11:15:00Z"},
]